In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.model_selection as sk
import tensorflow as tf

data = pd.read_csv("data/airfoil-noise.dat", sep="\t", header=None)
data = np.array(data)


C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Splitting into training and test
X_train, X_test, y_train, y_test = sk.train_test_split(data[:,:-1],data[:,-1],test_size=0.33, random_state = 42)

In [3]:
# Linear Regression
# Placeholder, Constants and Variables (if any)
X = tf.placeholder("float", shape=X_train.shape, name="train-x")
Y = tf.placeholder("float", shape=y_train.shape, name="train-y")
# learning_rate = tf.constant(0.01, dtype=tf.float32)
learning_rate = 0.01
epochs = 15
step = 10
W = tf.Variable(tf.random_normal([X_train.shape[1],1]), name="weights")
b = tf.Variable(np.random.randn(), name="bias")

In [4]:
# Our Hypothesis
hypothesis = tf.add(tf.matmul(X,W), b)

In [5]:
# Computing cost (mean squared error)
cost = tf.reduce_sum(tf.pow(hypothesis-Y, 2))/(2*X_train.shape[0])

In [6]:
# Optimizer
optimizer_grad = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
optimizer_adam = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(1, epochs):
        sess.run(optimizer_grad, feed_dict={X:X_train, Y:y_train})
        if epoch%step == 0:
            c = sess.run(cost, feed_dict={X:X_train,Y:y_train})
            print ("Epoch ", '%04d' % epoch, "Cost ", c)

Epoch  0010 Cost  nan
